In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
!pip install textrankr
!pip install konlpy
!pip install textrank

In [3]:
df = pd.read_csv('../data/train.csv')
df.head()

,docID,date,writerName,title,content,tag
0,1,20230214,머니S,"정부, AI반도체 석·박사 집중 육성… 대학당 '6년간 164억원' 지원",정부가 미국 AI 개발업체인 '오픈AI'(OpenAI)의 '챗GPT'(ChatGPT...,0
1,2,20230215,뉴시스,인사 청탁 대가 금품수수 의혹 전 소방청장 영장 기각,"기사내용 요약 법원 ""피의 사실 일부 다툼 여지, 불구속 상태 방어권 보장 필요"" ...",0
2,3,20230214,아이뉴스24,튀르키예 강진에 우리나라 지하수가 출렁였다,튀르키예에서 발생한 강진에 우리나라의 지하수가 출렁였다는 관측 보고가 나왔다. 한국...,0
3,4,20230215,데일리안,"멸치쇼핑, 2023년 신입 및 경력 사원 대규모 공채 진행",[데일리안 = 박영민 기자] 오픈마켓 멸치쇼핑이 2023년 신입 및 경력 사원을 대...,0
4,5,20230111,뉴스1,"美국방부, 추모의 벽 전사자 명단 오류에 ""유감스러운 실수""","국방부 대변인 ""실수 바로잡기 위해 내무부와 협력""…'오류 발견' 가족 등에 연락 ...",1


### **Summarize**

In [4]:
from typing import List
# from eunjeon import Mecab
from konlpy.tag import Okt


# class NounsTokenizer:
#     mecab = Mecab()
#     def __call__(self, df):
#         tokens = df['content'].apply(lambda x: mecab.nouns(x))
#         return tokens

class OktTokenizer:
    okt: Okt = Okt()

    def __call__(self, text: str) -> List[str]:
        tokens: List[str] = self.okt.phrases(text)
        return tokens

In [5]:
from typing import List
from textrankr import TextRank

myTokenizer = OktTokenizer()
textrank = TextRank(myTokenizer)

k: int = 3  # num sentences in the resulting summary

text = df['content'][0]
# summarized = textrank.summarize(text, k)
# print(summarized)  # gives you some text

# if verbose = False, it returns a list
summaries: List[str] = textrank.summarize(text, k, verbose=False)
for summary in summaries:
    print(summary)

정부가 미국 AI 개발업체인 '오픈AI'(OpenAI)의 '챗GPT'(ChatGPT)의 등장으로 AI 산업 시장에 일어날 대격변에 앞서 AI반도체 산업 고급 인재양성을 위한 대학원 사업 공고를 시행한다
정부는 이번 사업을 통해 국가 전략기술이자 경제 안보의 핵심 품목인 AI반도체 분야 기술 경쟁력을 높이고 미래 유망 신시장을 창출, 세계적인 수준의 AI반도체 설계와 AI소프트웨어 전문 고급인재 약 500명 양성을 목표로 하고 있다
전영수 과기부 정보통신산업정책관은 "데이터 센터와 자율주행, 사물인터넷 등 AI 산업 확산에 따라 다양한 분야로 AI반도체 활용이 확대되고 있다"며 "AI반도체 대학원 등 디지털 첨단 분야 인재양성을 통해 미래 유망 분야 산업 생태계 조성을 더욱 공고히 하고 국가 기술 주권 확보를 위해 총력을 다하겠다"고 말했다


### **Keyword Extraction**
- 참고 : https://lovit.github.io/nlp/2019/04/30/textrank/


In [6]:
from collections import Counter

def scan_vocabulary(sents, tokenize, min_count=2):
    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w:c for w,c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

In [7]:
from scipy.sparse import csr_matrix

def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

In [8]:
from collections import defaultdict

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 9:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

In [9]:
def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=2):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    tokens = [tokenize(sent) for sent in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence)
    return g, idx_to_vocab

In [10]:
import numpy as np
from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

In [11]:

from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenize(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

def textrank_keyword(sents, tokenize, min_count, window, min_cooccurrence, df=0.85, max_iter=30, topk=30):
    g, idx_to_vocab = word_graph(sents, tokenize, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

In [12]:
## Test
# keywords = textrank_keyword(
#     summaries,
#     tokenize = komoran_tokenize,
#     min_count = 2, 
#     window = 2,
#     min_cooccurrence=2
# )

In [13]:
# NaN 있는 행 제거
print('NaN이 있는 행 개수 : ', df['content'].isna().sum())
df_dropna = df.dropna(axis=0, subset=['content']) # content가 없는 기사는 drop --> 총 102개
df_dropna['title_content'] = df_dropna['title'] + ' ' + df_dropna['content'] # title + content
df_dropna['title_content'].head()

text = df_dropna['content'].apply(lambda x: x.split('. '))

NaN이 있는 행 개수 :  102


C:\Users\vaiv\AppData\Local\Temp\ipykernel_9296\826635131.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropna['title_content'] = df_dropna['title'] + ' ' + df_dropna['content'] # title + content


In [16]:
keywords_list = []
for idx, sents in tqdm(enumerate(text)):
    try:
        keywords = textrank_keyword(
            sents,
            tokenize = komoran_tokenize,
            min_count = 2, 
            window = 2,
            min_cooccurrence=2
        )
        keywords = [x for x in keywords if len(x[0].split('/')[0])>1]
        len_words = min(len(keywords), 10)
        keywords_list.append(keywords[:len_words])
    except Exception as e:
        # print(idx, ' ', e)
        keywords_list.append('')

7898it [03:04, 42.71it/s]


In [20]:
keyword = [[y[0].split('/')[0] for y in x] for x in keywords_list]
scores = [[y[1] for y in x] for x in keywords_list]

In [23]:
# csv 파일로 저장
result = pd.read_csv('../data/train_result.csv')
df_dropna['textrank'] = keyword
df_dropna['textrank_score'] = scores
result = pd.merge(result, df_dropna[['docID', 'textrank', 'textrank_score']], on='docID', how='left')
result.to_csv('../data/train_result.csv', index=False, encoding='utf-8-sig')

C:\Users\vaiv\AppData\Local\Temp\ipykernel_9296\412472281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropna['textrank'] = keyword
C:\Users\vaiv\AppData\Local\Temp\ipykernel_9296\412472281.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropna['textrank_score'] = scores


### ***...Test...***

In [47]:
text = '''윤석열 대통령은 8일 국민의힘 차기 지도부를 선출하는 전당대회에 참석해 “나라의 위기 그리고 당의 위기를 정치적 기회로 악용하면 절대 안 된다”고 강조했다. 윤 대통령은 이날 경기 고양시 일산 킨텍스에서 열린 국민의힘 제3차 전당대회에서 축사를 통해 “우리는 어떠한 부당한 세력과도 (싸우는 것을) 주저하지 말고 두려워하지 말아야 한다. 그것이 우리 당이 국민으로부터 더욱 사랑받는 길”이라며 이같이 밝혔다. 윤 대통령은 정진석 비상대책위원장과 주호영 원내대표 등 국민의힘 지도부에게 “어려운 시기에 당을 이끌어줘서 고생 많았다”고 인사한 뒤 “새롭게 선출될 국민의힘 지도부에 먼저 뜨거운 격려의 박수를 보낸다”고 했다. 이어 국민의힘이 보수정당으로서, 집권 여당으로서 나아가야 할 지향점을 제시했다. 윤 대통령은 “우리 당은 우리 번영의 토대인 자유민주주의 헌법 가치를 수호하는 정당으로서 약자를 따뜻하게 배려하는 사회를 만들어갈 책임이 있다”며 “국제사회의 리더 국가로서 우리와 세계 시민의 자유를 확장하고 국제사회의 평화와 번영을 만들어갈 책임이 있다”고 말했다. 이어 “새로 선출될 지도부와 우리 모두 하나가 돼야 한다”며 “우리 국민의힘 당내 선거에서는 승자도 패자도 없다”고 강조했다. 그러면서 “우리 당 구성원 모두 첫째도 국민, 둘째도 국민, 셋째도 국민만을 생각하고 함께 전진해야 한다”며 “우리 모두 힘을 합쳐 다시 대한민국, 새로운 국민의 나라를 만들어 가자”고 했다. 윤석열 대통령이 8일 경기 고양시 킨텍스에서 열린 국민의힘 제3차 전당대회에서 축사를 하고 있다. 윤 대통령은 새 정부 출범 후 10개월을 되돌아보며 “저는 그동안 우리의 헌법 정신인 자유와 연대의 가치를 역설해왔다. 이제 우리는 무너진 자유민주주의와 시장경제 체제를 바로잡아 달라는 국민의 목소리, 강력한 국가 안보를 통해 평화를 지켜달라는 국민의 목소리에 더 강력하게 행동하고 더 신속하게 실천해야 한다”고 말했다. 이어 “시장이 자유롭고 공정하게 작동할 수 있도록 우리의 제도를 선진 글로벌 스탠더드에 맞추고, 국민을 고통에 빠뜨리는 기득권 이권 카르텔은 확실하게 뿌리 뽑아야 한다”며 “우리 사회의 지속 가능성과 청년세대를 위한 노동, 교육, 연금 3대 개혁을 흔들림 없이 추진해야 한다”고 했다. 윤 대통령은 “우리의 미래는 결코 저절로 오지 않는다”며 “기득권의 집요한 저항에 부딪혀도 미래세대를 위한 길, 나라의 혁신을 위한 길을 결코 포기하거나 늦춰서는 안 된다”고 강조했다. 윤 대통령은 이날 축사를 위해 오른 단상에서 어퍼컷 세리머니를 선보이기도 했다. 현직 대통령이 집권 여당 전당대회에 참석한 것은 2008년 이명박 전 대통령(한나라당), 2014년 박근혜 전 대통령(새누리당) 이후 9년 만이다.'''

In [48]:
sents = text.split('. ')
sents

['윤석열 대통령은 8일 국민의힘 차기 지도부를 선출하는 전당대회에 참석해 “나라의 위기 그리고 당의 위기를 정치적 기회로 악용하면 절대 안 된다”고 강조했다',
 '윤 대통령은 이날 경기 고양시 일산 킨텍스에서 열린 국민의힘 제3차 전당대회에서 축사를 통해 “우리는 어떠한 부당한 세력과도 (싸우는 것을) 주저하지 말고 두려워하지 말아야 한다',
 '그것이 우리 당이 국민으로부터 더욱 사랑받는 길”이라며 이같이 밝혔다',
 '윤 대통령은 정진석 비상대책위원장과 주호영 원내대표 등 국민의힘 지도부에게 “어려운 시기에 당을 이끌어줘서 고생 많았다”고 인사한 뒤 “새롭게 선출될 국민의힘 지도부에 먼저 뜨거운 격려의 박수를 보낸다”고 했다',
 '이어 국민의힘이 보수정당으로서, 집권 여당으로서 나아가야 할 지향점을 제시했다',
 '윤 대통령은 “우리 당은 우리 번영의 토대인 자유민주주의 헌법 가치를 수호하는 정당으로서 약자를 따뜻하게 배려하는 사회를 만들어갈 책임이 있다”며 “국제사회의 리더 국가로서 우리와 세계 시민의 자유를 확장하고 국제사회의 평화와 번영을 만들어갈 책임이 있다”고 말했다',
 '이어 “새로 선출될 지도부와 우리 모두 하나가 돼야 한다”며 “우리 국민의힘 당내 선거에서는 승자도 패자도 없다”고 강조했다',
 '그러면서 “우리 당 구성원 모두 첫째도 국민, 둘째도 국민, 셋째도 국민만을 생각하고 함께 전진해야 한다”며 “우리 모두 힘을 합쳐 다시 대한민국, 새로운 국민의 나라를 만들어 가자”고 했다',
 '윤석열 대통령이 8일 경기 고양시 킨텍스에서 열린 국민의힘 제3차 전당대회에서 축사를 하고 있다',
 '윤 대통령은 새 정부 출범 후 10개월을 되돌아보며 “저는 그동안 우리의 헌법 정신인 자유와 연대의 가치를 역설해왔다',
 '이제 우리는 무너진 자유민주주의와 시장경제 체제를 바로잡아 달라는 국민의 목소리, 강력한 국가 안보를 통해 평화를 지켜달라는 국민의 목소리에 더 강력하게 행동하고 더 신속하게 실천해야 한다”고 말했다',
 '이어 “시장이 

In [49]:
keywords = textrank_keyword(
    sents,
    tokenize = komoran_tokenize,
    min_count = 2, 
    window = 2,
    min_cooccurrence=2
)
keywords = [x for x in keywords if len(x[0].split('/')[0])>1]

In [50]:
keywords

[('국민/NNG', 3.3973521725045885),
 ('대통령/NNG', 2.980467999842943),
 ('대회/NNG', 1.359786218246375),
 ('전당/NNG', 1.359786218246375),
 ('만들/VV', 1.1961659729234588),
 ('사회/NNG', 1.1924562844477715),
 ('지도부/NNG', 1.0467066708470085),
 ('있다/NNP', 0.993454661260788),
 ('책임/NNG', 0.993454661260788),
 ('번영/NNG', 0.993454661260788),
 ('위하/VV', 0.9663758575334535),
 ('나라/NNG', 0.9300298068989508),
 ('축사/NNG', 0.8889241028078108),
 ('자유/NNG', 0.8569376887155538),
 ('강조/NNG', 0.8522070359249805),
 ('자유민주주의/NNP', 0.8453184426620151),
 ('국가/NNG', 0.8453184426620151),
 ('평화/NNG', 0.8453184426620151),
 ('선출/NNG', 0.8198452086379223),
 ('위기/NNG', 0.7946872292811474),
 ('경기/NNP', 0.7799493685485686),
 ('고양시/NNP', 0.7799493685485686),
 ('킨텍스/NNP', 0.7799493685485686)]

In [51]:
from typing import List
from textrankr import TextRank

myTokenizer = OktTokenizer()
textrank = TextRank(myTokenizer)

k: int = 3  # num sentences in the resulting summary

summarized = textrank.summarize(text, k)
print(summarized)  # gives you some text

윤석열 대통령은 8일 국민의힘 차기 지도부를 선출하는 전당대회에 참석해 “나라의 위기 그리고 당의 위기를 정치적 기회로 악용하면 절대 안 된다”고 강조했다
윤 대통령은 이날 경기 고양시 일산 킨텍스에서 열린 국민의힘 제3차 전당대회에서 축사를 통해 “우리는 어떠한 부당한 세력과도 (싸우는 것을) 주저하지 말고 두려워하지 말아야 한다
윤석열 대통령이 8일 경기 고양시 킨텍스에서 열린 국민의힘 제3차 전당대회에서 축사를 하고 있다
